In [2]:
api_key='RGAPI-e8d88463-69fe-4608-a71f-542b86e57ac0'
import requests
import time

In [3]:
def get_match_info_at_twenty(matchId=None):
    link = link= f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('Rate limit exceeded')
            time.sleep(10)
            continue
        elif response.status_code==200:
            break
        elif response.status_code==404:
            return 0
        else:
            print('Response is bad: '+str(response.status_code))
    match_data = response.json()

    if match_data['info']['queueId'] ==420:
        timeline_api_link = f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline?api_key={api_key}'

        while True:
            timeline_response = requests.get(timeline_api_link)
            if timeline_response.status_code==429:
                print('Rate limit exceeded')
                time.sleep(10)
                continue
            elif timeline_response.status_code==200:
                break
            elif timeline_response.status_code==404:
                return 0
            else:
                print('Response is bad: '+str(timeline_response.status_code))
        timeline_info = timeline_response.json()
        return timeline_info
    
    else:
        print('not a ranked match')
        return 0
            


In [4]:
get_match_info_at_twenty(matchId='EUW1_6928661440')

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6928661440',
  'participants': ['ymuTPiZBHdEb3SSB9ZbHVc-uozTSymjCqK7jfJqtZqysu9-c4ZdGLGxo2b_2w1VxX3Fb_uOznejzCg',
   'FsxJc00eXGlpaLeyPzvc1oACxy2nUPVmZ-43b-Y8TUMuMPZlBHzwlVd0C--WfT3sSqHigmrNx3UvwQ',
   'qCEpewevkAgS4MiUZjaXfpUDB_EhepM_JiEnCXuHTnMN0ibUPpD0dJmCpOe0pGX4RHT0zQZRrqI0FA',
   'YoSMi89utVQJehI5ZmS_tJ35Di2MP5ghbWJOBHflrXPXGM21ifB8N5LB10FKKYdyMWiLK8Gq_hRCuw',
   'D-59Oxm3IXmVb58SnEuKLO6-bTLrgBK8aEvhGSU8s5LF5LP3k5TpBJ2uZZvh-kxO5VEdTg1P5ji45Q',
   'FBlRkyWjuLrQXlw9n5ECFtbkYcLlA0zdlYmDxefW_LgqesSyvwn8w9rZ8ws4AeosMmxQ-RoGKpT9hQ',
   'buvMukA-JVre-d3tH8WGdH0Fewk5VwFKmuVog33gO7PDRcf_-BV4Iy6jCV0DsaN_HfPU1Or2suJVSA',
   'wXDXfxY2_sDY5SqZFm1FPFkS3GKSG0TtpaAikXzMAYCKrSG8rl5upVeZRhKvlvHmGAciNEgMPatPvw',
   'o1RiwtCHVQdt1ut0W2UK-RSe02NRSNSLBfgEgBwRq46MWr8l3Mk5nEL_CnP1AuH6fStS7-Fg5f_Ahg',
   'zkLpCEh4CJFmpaB6s8xcz7QrD3mYaYBCa-CL1_qU5H7d0ubiDoOxE2UrFhCDMjux7PIV2ezroU-mTw']},
 'info': {'endOfGameResult': 'GameComplete',
  'frameInterval': 60

In [7]:
import pandas as pd

In [18]:
def get_specific_match_stats_at_twenty(data):
    blue_side = data['info']['participants'][:5]
    red_side = data['info']['participants'][5:]

    #blue side statistics
    #we will not consider baron in this because it spawns at exactly 20 minutes so it's impossible for any team to kill it in time
    total_blue_side_minions = 0
    total_blue_side_kills=0
    total_blue_side_gold_earned = 0
    total_blue_side_wards_placed = 0
    total_blue_side_wards_killed = 0
    total_towers_destroyed_by_blue = 0
    total_inhibitors_destroyed_by_blue =0
    total_blue_side_dragons_killed =0
    total_blue_side_riftHeralds_killed =0
    blue_team_first_blood =0
    blue_side_won=0

    #red side statistics
    total_red_side_minions = 0
    total_red_side_kills=0
    total_red_side_gold_earned = 0
    total_red_side_wards_placed = 0
    total_red_side_wards_killed = 0
    total_towers_destroyed_by_red = 0
    total_inhibitors_destroyed_by_red =0
    total_red_side_dragons_killed =0
    total_red_side_riftHeralds_killed =0
 


    if len(data['info']['frames']) < 21:
        return 0
    

    #getting if blue side won or not (target variable)    
    for event in data['info']['frames'][-1]['events']:
        if event['type'] == 'GAME_END':
            if event['winningTeam'] ==100:
                blueWin+=1
    
    for participant in blue_side:
        total_blue_side_minions += data['info']['frames'][20]['participantFrames'][str(participant['participantId'])]['minionsKilled']
        total_blue_side_gold_earned+= data['info']['frames'][20]['participantFrames'][str(participant['participantId'])]['totalGold']

    for participant in red_side:
        total_red_side_minions += data['info']['frames'][20]['participantFrames'][str(participant['participantId'])]['minionsKilled']
        total_red_side_gold_earned+= data['info']['frames'][20]['participantFrames'][str(participant['participantId'])]['totalGold']


    #for loop to iterate over the events that occurred at the first 20 minutes/'frames' in the game
    for frame in range(21):
        for event in data['info']['frames'][frame]['events']:
            if event['type']=='WARD_PLACED':
                if event['creatorId'] in range(1,6):
                    total_blue_side_wards_placed+=1
                else:
                    total_red_side_wards_placed+=1
            if event['type']=='CHAMPION_KILL':
                if event['killerId'] in range(1,6):
                    total_blue_side_kills+=1
                else:
                    total_red_side_kills+=1






    df = pd.DataFrame({
        'total_blue_side_minions': [total_blue_side_minions],
        'total_blue_side_gold_earned': [total_blue_side_gold_earned],
        'total_red_side_minions':[total_red_side_minions],
        'total_red_side_gold_earned':[total_red_side_gold_earned],
        'total_blue_side_wards_placed':[total_blue_side_wards_placed],
        'total_blue_side_kills':[total_blue_side_kills],
        'total_red_side_kills':[total_red_side_kills],
        'total_red_side_wards_placed':[total_red_side_wards_placed]

    })

    return df
        



In [19]:
get_specific_match_stats_at_twenty(data=get_match_info_at_twenty(matchId='EUW1_6928661440'))

,total_blue_side_minions,total_blue_side_gold_earned,total_red_side_minions,total_red_side_gold_earned,total_blue_side_wards_placed,total_blue_side_kills,total_red_side_kills,total_red_side_wards_placed
0,506,34325,496,35824,49,11,13,43
